In [1]:
import requests
import json
import prettytable
from bs4 import BeautifulSoup as bs
import pymongo
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pygeoj

In [2]:
#setting up splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless = False)



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Get LATEST driver version for 94.0.4606
Trying to download new driver from https://chromedriver.storage.googleapis.com/94.0.4606.61/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\frank\.wdm\drivers\chromedriver\win32\94.0.4606.61]


In [3]:
table_url = "https://www.bls.gov/oes/current/oes_nat.htm"
tablehtml = pd.read_html(table_url)
tableraw = tablehtml[0]
tableraw = tableraw.drop([0, 1041])
tableraw = tableraw.rename(columns = {"Occupation title (click on the occupation title to view its profile)": "Occupation", 'Employment': 'Number_of_Jobs_Listed','Employment per 1,000 jobs': 'Job_Frequency', 'Annual mean wage': 'Salary'})

#table with grouped categories
maincategories = tableraw[tableraw['Occupation code'].str[3].isin(['0'])].reset_index(drop = True)


#cleaned up full table
cleanedtable = tableraw.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]
# cleanedtable
maincategories

,Occupation code,Occupation,Level,Number_of_Jobs_Listed,Employment RSE,Job_Frequency,Median hourly wage,Mean hourly wage,Salary,Mean wage RSE
0,00-0000,All Occupations,total,139099570,0.1%,1000.000,$20.17,$27.07,"$56,310",0.1%
1,11-0000,Management Occupations,major,7947300,0.2%,57.134,$52.77,$60.81,"$126,480",0.2%
2,13-0000,Business and Financial Operations Occupations,major,8387490,0.2%,60.298,$34.73,$38.79,"$80,680",0.2%
3,15-0000,Computer and Mathematical Occupations,major,4587700,0.5%,32.981,$43.92,$46.53,"$96,770",0.5%
4,17-0000,Architecture and Engineering Occupations,major,2515040,0.5%,18.081,$39.98,$43.41,"$90,300",0.3%
5,19-0000,"Life, Physical, and Social Science Occupations",major,1296060,0.7%,9.317,$33.54,$38.15,"$79,360",0.4%
6,21-0000,Community and Social Service Occupations,major,2231070,0.5%,16.039,$22.85,$25.09,"$52,180",0.4%
7,23-0000,Legal Occupations,major,1154740,0.5%,8.302,$40.82,$54.00,"$112,320",0.7%
8,25-0000,Educational Instruction and Library Occupations,major,8446910,0.4%,60.726,$25.18,$28.75,"$59,810",0.5%
9,27-0000,"Arts, Design, Entertainment, Sports, and Media...",major,1857500,0.7%,13.354,$25.55,$30.96,"$64,400",0.7%


In [4]:
management_occupations = tableraw[tableraw['Occupation code'].str.contains('11-')].reset_index(drop = True)
management_occupations = management_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

business_occupations = tableraw[tableraw['Occupation code'].str.contains('13-')].reset_index(drop = True)
business_occupations = business_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

computer_occupations = tableraw[tableraw['Occupation code'].str.contains('15-')].reset_index(drop = True)
computer_occupations = computer_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

engineering_occupations = tableraw[tableraw['Occupation code'].str.contains('17-')].reset_index(drop = True)
engineering_occupations = engineering_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

science_occupations = tableraw[tableraw['Occupation code'].str.contains('19-')].reset_index(drop = True)
science_occupations = science_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

social_occupations = tableraw[tableraw['Occupation code'].str.contains('21-')].reset_index(drop = True)
social_occupations = social_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

legal_occupations = tableraw[tableraw['Occupation code'].str.contains('23-')].reset_index(drop = True)
legal_occupations = legal_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

education_occupations = tableraw[tableraw['Occupation code'].str.contains('25-')].reset_index(drop = True)
education_occupations = education_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

arts_occupations = tableraw[tableraw['Occupation code'].str.contains('27-')].reset_index(drop = True)
arts_occupations = arts_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

healthcare_occupations = tableraw[tableraw['Occupation code'].str.contains('29-')].reset_index(drop = True)
healthcare_occupations = healthcare_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

healthcaresup_occupations = tableraw[tableraw['Occupation code'].str.contains('31-')].reset_index(drop = True)
healthcaresup_occupations = healthcaresup_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

protective_occupations = tableraw[tableraw['Occupation code'].str.contains('33-')].reset_index(drop = True)
protective_occupations = protective_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

food_occupations = tableraw[tableraw['Occupation code'].str.contains('35-')].reset_index(drop = True)
food_occupations = food_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

building_occupations = tableraw[tableraw['Occupation code'].str.contains('37-')].reset_index(drop = True)
building_occupations = building_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

personalcare_occupations = tableraw[tableraw['Occupation code'].str.contains('39-')].reset_index(drop = True)
personalcare_occupations = personalcare_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

sales_occupations = tableraw[tableraw['Occupation code'].str.contains('41-')].reset_index(drop = True)
sales_occupations = sales_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

office_occupations = tableraw[tableraw['Occupation code'].str.contains('43-')].reset_index(drop = True)
office_occupations = office_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

farming_occupations = tableraw[tableraw['Occupation code'].str.contains('45-')].reset_index(drop = True)
farming_occupations = farming_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

construction_occupations = tableraw[tableraw['Occupation code'].str.contains('47-')].reset_index(drop = True)
construction_occupations = construction_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

maintenance_occupations = tableraw[tableraw['Occupation code'].str.contains('49-')].reset_index(drop = True)
maintenance_occupations = maintenance_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

production_occupations = tableraw[tableraw['Occupation code'].str.contains('51-')].reset_index(drop = True)
production_occupations = production_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

transportation_occupations = tableraw[tableraw['Occupation code'].str.contains('53-')].reset_index(drop = True)
transportation_occupations = transportation_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

# test = transportation_occupations.to_dict('records')
# test

In [5]:
csvpath = "../resources/choroplethedit.csv"
wagecsv = pd.read_csv(csvpath)
wagecsv

C:\Users\frank\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Area text,Occupation text,Job characteristic text,Average hourly wage\n($),Area code,Occupation \ncode,Job characteristic code,Work level code
0,National,Management occupations,All workers,24.68,0,110000,0,7
1,National,Management occupations,All workers,31.21,0,110000,0,8
2,National,Management occupations,All workers,36.84,0,110000,0,9
3,National,Management occupations,All workers,43.78,0,110000,0,10
4,National,Management occupations,All workers,56.76,0,110000,0,11
...,...,...,...,...,...,...,...,...
487049,Eastern Wyoming nonmetropolitan area,Stockers and order fillers,Full-time,16.18,5600007,537065,25,2
487050,Eastern Wyoming nonmetropolitan area,Stockers and order fillers,Full-time,18.56,5600007,537065,25,3
487051,Eastern Wyoming nonmetropolitan area,Stockers and order fillers,Part-time,12.29,5600007,537065,26,0
487052,Eastern Wyoming nonmetropolitan area,Stockers and order fillers,Part-time,13.84,5600007,537065,26,2


In [6]:
# wagecsv['Occupation text'].value_counts()

In [7]:
# nonational['Occupation text'].nunique()

In [8]:
# list=[ i for i in range(100)]
# base_path="resources/geojson"
# for i in range(list):
#     path=f"{base_path}{list[i]}.json"


In [9]:
nonational = wagecsv.loc[wagecsv['Area text'] != 'National']
df=nonational.sort_values('Occupation text')
df
df1=df[df['Area text'].str.contains(", CA")]
df2=df[df['Area text'].str.contains("California")]
df3=df2[df2['Area text']!='California-Lexington Park, MD']
dfca=pd.concat([df1, df3])
dfca['Area text']="CA"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", AL")]
df2=df[df['Area text'].str.contains("Alabama")]
dfal=pd.concat([df1, df2])
dfal['Area text']="AL"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", AK")]
df2=df[df['Area text'].str.contains("Alaska")]
dfak=pd.concat([df1, df2])
dfak['Area text']="AK"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", AZ")]
df2=df[df['Area text'].str.contains("Arizona")]
dfaz=pd.concat([df1, df2])
dfaz['Area text']="AZ"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", AR")]
df2=df[df['Area text'].str.contains("Arkansas")]
dfar=pd.concat([df1, df2])
dfar['Area text']="AR"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", CO")]
df2=df[df['Area text'].str.contains("Colorado")]
dfco=pd.concat([df1, df2])
dfco['Area text']="CO"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", CT")]
df2=df[df['Area text'].str.contains("Connecticut")]
dfct=pd.concat([df1, df2])
dfct['Area text']="CT"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", DE")]
df2=df[df['Area text'].str.contains("Delaware")]
dfde=pd.concat([df1, df2])
dfde['Area text']="DE"
# dfca['Occupation text'].value_counts().reset_index()

dfdc=df[df['Area text'].str.contains("District of Columbia")]
dfdc['Area text']="DC"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", FL")]
df2=df[df['Area text'].str.contains("Florida")]
dffl=pd.concat([df1, df2])
dffl['Area text']="FL"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", GA")]
df2=df[df['Area text'].str.contains("Georgia")]
dfga=pd.concat([df1, df2])
dfga['Area text']="GA"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", HI")]
df2=df[df['Area text'].str.contains("Hawaii")]
dfhi=pd.concat([df1, df2])
dfhi['Area text']="HI"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", ID")]
df2=df[df['Area text'].str.contains("Idaho")]
dfid=pd.concat([df1, df2])
dfid['Area text']="ID"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", IL")]
df2=df[df['Area text'].str.contains("Illinois")]
dfil=pd.concat([df1, df2])
dfil['Area text']="IL"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", IN")]
df2=df[df['Area text'].str.contains("Indiana")]
df3=df2[df2['Area text']!='Indianapolis-Carmel-Anderson, IN']
dfin=pd.concat([df1, df3])
dfin['Area text']="IN"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", IA")]
df2=df[df['Area text'].str.contains("Iowa")]
df3=df2[df2['Area text']!="Iowa City, IA"]
dfia=pd.concat([df1, df3])
dfia['Area text']="IA"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", KS")]
df2=df[df['Area text'].str.contains("Kansas")]
dfks=pd.concat([df1, df2])
dfks['Area text']="KS"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", KY")]
df2=df[df['Area text'].str.contains("Kentucky")]
dfky=pd.concat([df1, df2])
dfky['Area text']="KY"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", LA")]
df2=df[df['Area text'].str.contains("Louisiana")]
dfla=pd.concat([df1, df2])
dfla['Area text']="LA"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", ME")]
df2=df[df['Area text'].str.contains("Maine")]
dfme=pd.concat([df1, df2])
dfme['Area text']="ME"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", MD")]
df2=df[df['Area text'].str.contains("Maryland")]
dfmd=pd.concat([df1, df2])
dfmd['Area text']="MD"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", MA")]
df2=df[df['Area text'].str.contains("Massachusetts")]
dfma=pd.concat([df1, df2])
dfma['Area text']="MA"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", MI")]
df2=df[df['Area text'].str.contains("Michigan")]
df3=df2[df2['Area text']!='Michigan City-La Porte, IN']
dfmi=pd.concat([df1, df3])
dfmi['Area text']="MI"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", MN")]
df2=df[df['Area text'].str.contains("Minnesota")]
dfmn=pd.concat([df1, df2])
dfmn['Area text']="MN"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", MS")]
df2=df[df['Area text'].str.contains("Mississippi")]
dfms=pd.concat([df1, df2])
dfms['Area text']="MS"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", MO")]
df2=df[df['Area text'].str.contains("Missouri")]
dfmo=pd.concat([df1, df2])
dfmo['Area text']="MO"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", MT")]
df2=df[df['Area text'].str.contains("Montana")]
dfmt=pd.concat([df1, df2])
dfmt['Area text']="MT"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", NE")]
df2=df[df['Area text'].str.contains("Nebraska")]
dfne=pd.concat([df1, df2])
dfne['Area text']="NE"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", NV")]
df2=df[df['Area text'].str.contains("Nevada")]
dfnv=pd.concat([df1, df2])
dfnv['Area text']="NV"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", NH")]
df2=df[df['Area text'].str.contains("New Hampshire")]
dfnh=pd.concat([df1, df2])
dfnh['Area text']="NH"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", NJ")]
df2=df[df['Area text'].str.contains("New Jersey")]
dfnj=pd.concat([df1, df2])
dfnj['Area text']="NJ"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", NM")]
df2=df[df['Area text'].str.contains("New Mexico")]
dfnm=pd.concat([df1, df2])
dfnm['Area text']="NM"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", NY")]
df2=df[df['Area text'].str.contains("New York")]
df3=df2[df2['Area text']!='New York-Newark-Jersey City, NY-NJ-PA']
dfny=pd.concat([df1, df3])
dfny['Area text']="NY"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", NC")]
df2=df[df['Area text'].str.contains("North Carolina")]
dfnc=pd.concat([df1, df2])
dfnc['Area text']="NC"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", ND")]
df2=df[df['Area text'].str.contains("North Dakota")]
dfnd=pd.concat([df1, df2])
dfnd['Area text']="ND"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", OH")]
df2=df[df['Area text'].str.contains("Ohio")]
dfoh=pd.concat([df1, df2])
dfoh['Area text']="OH"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", OK")]
df2=df[df['Area text'].str.contains("Oklahoma")]
df3=df2[df2['Area text']!='Oklahoma City, OK']
dfok=pd.concat([df1, df3])
dfok['Area text']="OK"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", OR")]
df2=df[df['Area text'].str.contains("Oregon")]
dfor=pd.concat([df1, df2])
dfor['Area text']="OR"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", PA")]
df2=df[df['Area text'].str.contains("Pennsylvania")]
dfpa=pd.concat([df1, df2])
dfpa['Area text']="PA"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", RI")]
df2=df[df['Area text'].str.contains("Rhode Island")]
dfri=pd.concat([df1, df2])
dfri['Area text']="RI"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", SC")]
df2=df[df['Area text'].str.contains("South Carolina")]
dfsc=pd.concat([df1, df2])
dfsc['Area text']="SC"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", SD")]
df2=df[df['Area text'].str.contains("South Dakota")]
dfsd=pd.concat([df1, df2])
dfsd['Area text']="SD"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", TN")]
df2=df[df['Area text'].str.contains("Tennessee")]
dftn=pd.concat([df1, df2])
dftn['Area text']="TN"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", TX")]
df2=df[df['Area text'].str.contains("Texas")]
dftx=pd.concat([df1, df2])
dftx['Area text']="TX"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", UT")]
df2=df[df['Area text'].str.contains("Utah")]
dfut=pd.concat([df1, df2])
dfut['Area text']="UT"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", VT")]
df2=df[df['Area text'].str.contains("Vermont")]
dfvt=pd.concat([df1, df2])
dfvt['Area text']="VT"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", VA")]
df2=df[df['Area text'].str.contains("Virginia")]
df3=df2[df2['Area text']!='Virginia Beach-Norfolk-Newport News, VA-NC']
df4=df3[df3['Area text']!='Northern West Virginia nonmetropolitan area']
df5=df4[df4['Area text']!='Southern West Virginia nonmetropolitan area']
df6=df5[df5['Area text']!='West Virginia']
dfva=pd.concat([df1, df6])
dfva['Area text']="VA"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", WA")]
df2=df[df['Area text'].str.contains("Washington")]
dfwa=pd.concat([df1, df2])
dfwa['Area text']="WA"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", WV")]
df2=df[df['Area text'].str.contains("West Virginia")]
dfwv=pd.concat([df1, df2])
dfwv['Area text']="WV"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", WI")]
df2=df[df['Area text'].str.contains("Wisconsin")]
dfwi=pd.concat([df1, df2])
dfwi['Area text']="WI"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", WY")]
df2=df[df['Area text'].str.contains("Wyoming")]
df3=df2[df2['Area text']!='Grand Rapids-Wyoming, MI']
dfwy=pd.concat([df1, df3])
dfwy['Area text']="WY"
# dfca['Occupation text'].value_counts().reset_index()

<ipython-input-9-0d471e261f3f>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfdc['Area text']="DC"


In [10]:
# CONCAT FILE
bigboi = pd.concat([dfal, dfak, dfaz, dfar, dfca, dfco, dfct, dfde, dfdc, dffl, dfga, dfhi, dfid, dfil, dfin, dfia, dfks, dfky, dfla, dfme, dfmd, dfma, dfmi, dfmn, dfms, dfmo, dfmt, dfne, dfnv, dfnh, dfnj, dfnm, dfny, dfnc, dfnd, dfoh, dfok, dfor, dfpa, dfri, dfsc, dfsd, dftn, dftx, dfut, dfvt, dfva, dfwa, dfwv, dfwi, dfwy])
bigboi

,Area text,Occupation text,Job characteristic text,Average hourly wage\n($),Area code,Occupation \ncode,Job characteristic code,Work level code
17210,AL,Accountants and auditors,All workers,22.91,11500,132011,0,7
17211,AL,Accountants and auditors,All workers,34.88,11500,132011,0,9
17212,AL,Accountants and auditors,All workers,28.10,11500,132011,0,16
17213,AL,Accountants and auditors,Nonunion,29.76,11500,132011,24,0
17217,AL,Accountants and auditors,Full-time,29.45,11500,132011,25,16
...,...,...,...,...,...,...,...,...
486981,WY,"Welders, cutters, solderers, and brazers",All workers,34.62,5600007,514121,0,6
486983,WY,"Welders, cutters, solderers, and brazers",Full-time,29.55,5600007,514121,25,0
485424,WY,"Welders, cutters, solderers, and brazers",Time-based pay,28.75,5600000,514121,27,0
485423,WY,"Welders, cutters, solderers, and brazers",Full-time,28.75,5600000,514121,25,0


In [11]:
occupation_list = nonational['Occupation text'].unique()
# for x in occupation_list:
    
#     print(x)
len(occupation_list)

469

In [12]:
bgroup=bigboi.groupby(['Occupation text','Area text'])
# bgroup["Job characteristic code"].count().reset_index().to_csv("../resources/IgotchuAndrew_sj.csv")
bggroup2 = bgroup["Job characteristic code"].count().reset_index().sort_values('Occupation text')
bggroup2

,Occupation text,Area text,Job characteristic code
0,Accountants and auditors,AK,33
28,Accountants and auditors,ND,55
29,Accountants and auditors,NE,62
30,Accountants and auditors,NH,51
31,Accountants and auditors,NJ,49
...,...,...,...
11554,Writers and authors,KS,4
11555,Writers and authors,LA,1
11556,Writers and authors,MO,7
11557,Writers and authors,NE,6


In [13]:
states = [dfal, dfak, dfaz, dfar, dfca, dfco, dfct, dfde, dfdc, dffl, dfga, dfhi, dfid, dfil, dfin, dfia, dfks, dfky, dfla, dfme, dfmd, dfma, dfmi, dfmn, dfms, dfmo, dfmt, dfne, dfnv, dfnh, dfnj, dfnm, dfny, dfnc, dfnd, dfoh, dfok, dfor, dfpa, dfri, dfsc, dfsd, dftn, dftx, dfut, dfvt, dfva, dfwa, dfwv, dfwi, dfwy]    
teststate = dfal.groupby(['Occupation text'])  
teststate2 = teststate["Job characteristic code"].count().reset_index().sort_values('Occupation text').set_index('Occupation text')
testdict = teststate2.to_dict()
testdict

{'Job characteristic code': {'Accountants and auditors': 133,
  'Administrative services and facilities managers': 12,
  'Aircraft mechanics and service technicians': 4,
  'Animal caretakers': 3,
  'Architectural and civil drafters': 3,
  'Architectural and engineering managers': 9,
  'Architecture and engineering occupations': 205,
  'Arts, design, entertainment, sports, and media occupations': 108,
  'Automotive body and related repairers': 1,
  'Automotive service technicians and mechanics': 23,
  'Bakers': 3,
  'Bartenders': 6,
  'Bill and account collectors': 12,
  'Billing and posting clerks': 38,
  'Bookkeeping, accounting, and auditing clerks': 199,
  'Brokerage clerks': 5,
  'Building and grounds cleaning and maintenance occupations': 284,
  'Bus and truck mechanics and diesel engine specialists': 13,
  'Business and financial operations occupations': 275,
  'Buyers and purchasing agents': 63,
  'Carpenters': 15,
  'Cashiers': 252,
  'Chemical equipment operators and tenders':

In [14]:
states = [dfal, dfak, dfaz, dfar, dfca, dfco, dfct, dfde, dfdc, dffl, dfga, dfhi, dfid, dfil, dfin, dfia, dfks, dfky, dfla, dfme, dfmd, dfma, dfmi, dfmn, dfms, dfmo, dfmt, dfne, dfnv, dfnh, dfnj, dfnm, dfny, dfnc, dfnd, dfoh, dfok, dfor, dfpa, dfri, dfsc, dfsd, dftn, dftx, dfut, dfvt, dfva, dfwa, dfwv, dfwi, dfwy]    
# geojsonpath = '../resources/geojsons/states.geojson'
# geojson1 = pd.read_json(geojsonpath)
newtest = pygeoj.load('../resources/geojsons/states.geojson')


for x in states:
    teststate = x.groupby(['Occupation text'])  
    teststate2 = teststate["Job characteristic code"].count().reset_index().sort_values('Occupation text').set_index('Occupation text')
    testdict = teststate2.to_dict()
    print(testdict)
    for y in newtest:
        y.properties = testdict['Job characteristic code']
        y.properties['jobs']=testdict
newtest.save("../resources/geojsons/newtest.geojson")

{'Job characteristic code': {'Accountants and auditors': 133, 'Administrative services and facilities managers': 12, 'Aircraft mechanics and service technicians': 4, 'Animal caretakers': 3, 'Architectural and civil drafters': 3, 'Architectural and engineering managers': 9, 'Architecture and engineering occupations': 205, 'Arts, design, entertainment, sports, and media occupations': 108, 'Automotive body and related repairers': 1, 'Automotive service technicians and mechanics': 23, 'Bakers': 3, 'Bartenders': 6, 'Bill and account collectors': 12, 'Billing and posting clerks': 38, 'Bookkeeping, accounting, and auditing clerks': 199, 'Brokerage clerks': 5, 'Building and grounds cleaning and maintenance occupations': 284, 'Bus and truck mechanics and diesel engine specialists': 13, 'Business and financial operations occupations': 275, 'Buyers and purchasing agents': 63, 'Carpenters': 15, 'Cashiers': 252, 'Chemical equipment operators and tenders': 3, 'Child, family, and school social worker

In [15]:
newtest = pygeoj.load('../resources/geojsons/states.geojson')
# newtest.save("../resources/geojsons/newtest.geojson")
# for x in newtest:
#     print(x.geometry.type)
#     print(x.geometry.coordinates)
newtest[0].properties['density']

94.65

In [16]:
geojsonpath = '../resources/geojsons/states.geojson'
geojson = pd.read_json(geojsonpath)
geojson.features[0]['properties']['density']

94.65

In [17]:

testgeo = pygeoj.load('states.geojson')
for feature in testgeo:
    print(feature.geometry.type)
    print(feature.geometry.coordinates)


FileNotFoundError: [Errno 2] No such file or directory: 'states.geojson'

In [ ]:
testgeo.save("test.geojson")

In [ ]:
# Alabama = 0
# Alaska = 0
# Arizona = 0
# Arkansas = 0
# California = 0
# Colorado = 0
# Connecticut = 0
# Delaware = 0
# DistrictColumbia = 0
# Florida = 0
# Georgia = 0
# Hawaii = 0
# Idaho = 0
# Illinois = 0
# Indiana = 0
# Iowa =  0
# Kansas = 0
# Kentucky = 0
# Louisiana = 0
# Maine = 0
# Maryland = 0
# Massachusetts = 0
# Michigan = 0
# Minnesota = 0
# Mississippi = 0
# Missouri = 0
# Montana = 0
# Nebraska = 0
# Nevada = 0
# NewHampshire = 0
# NewJersey = 0
# NewMexico = 0
# NewYork = 0
# NorthCarolina = 0
# NorthDakota = 0
# Ohio = 0
# Oklahoma = 0
# Oregon = 0
# Pennsylvania = 0
# RhodeIsland = 0
# SouthCarolina = 0
# SouthDakota = 0
# Tennessee = 0
# Texas = 0
# Utah = 0
# Vermont = 0
# Virginia = 0
# Washington = 0
# WestVirginia = 0
# Wisconsin = 0
# Wyoming = 0